In [1]:
import os 
os.chdir("./drive/My Drive/colab/pytorch/my_code/poem")
!ls


data.ipynb  tang.npz


In [5]:
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 132kB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.17.3
    Uninstalling numpy-1.17.3:
      Successfully uninstalled numpy-1.17.3


## 处理文本

In [2]:
import numpy as np 
datas = np.load("tang.npz")
data = datas['data']
ix2word = datas['ix2word'].item()
word2ix = datas['word2ix'].item()

print(data)

## 数据再继续处理一下 全部弄成五言律诗 没有标点，也没有额外的标识符
poem_list = []
for each_row in data:
    each_list = []
    for i in each_row:
        word = ix2word[i] ## 得到一个具体的字了
        if (word == "</s>" or word == "<START>" or word == "。" or word == "<EOP>"):
            continue
        else :
            each_list.append(word)

    poem_list.append(each_list)

## 得到的诗全是逗号分割
print(len(poem_list))
print(poem_list[:100])

poem = []
for each_poem in poem_list:
  str_poem = "".join(each_poem)
  sentence_poem = str_poem.split("，") ## 这样就把每句诗 都弄到一个列表里面 每句诗都是字符串
  for each in sentence_poem:
    poem.append(each)

print(poem[:10])


## 提取五言律诗
five_poem = []
for each_poem in poem:
  if (len(each_poem) == 5) :
    five_poem.append(each_poem)

print(len(five_poem))
print(five_poem[:10])



[[8292 8292 8292 ...  846 7435 8290]
 [8292 8292 8292 ... 7878 7435 8290]
 [8292 8292 8292 ... 4426 7435 8290]
 ...
 [8292 8292 8292 ... 7739 7435 8290]
 [8292 8292 8292 ... 7290 7435 8290]
 [8292 8292 8292 ... 1294 7435 8290]]
57580
[['度', '门', '能', '不', '访', '，', '冒', '雪', '屡', '西', '东', '已', '想', '人', '如', '玉', '，', '遥', '怜', '马', '似', '骢', '乍', '迷', '金', '谷', '路', '，', '稍', '变', '上', '阳', '宫', '还', '比', '相', '思', '意', '，', '纷', '纷', '正', '满', '空'], ['逍', '遥', '东', '城', '隅', '，', '双', '树', '寒', '葱', '蒨', '广', '庭', '流', '华', '月', '，', '高', '阁', '凝', '余', '霰', '杜', '门', '非', '养', '素', '，', '抱', '疾', '阻', '良', '䜩', '孰', '谓', '无', '他', '人', '，', '思', '君', '岁', '云', '变', '官', '曹', '亮', '先', '忝', '，', '陈', '躅', '慙', '俊', '彥', '岂', '知', '晨', '与', '夜', '，', '相', '代', '不', '相', '见', '缄', '书', '问', '所', '如', '，', '詶', '藻', '当', '芬', '绚'], ['川', '上', '风', '雨', '来', '，', '须', '臾', '满', '城', '阙', '岧', '峣', '青', '莲', '界', '，', '萧', '条', '孤', '兴', '发', '前', '山', '遽', '已', '净', '，', '阴', '霭', '夜', 

In [3]:
## 将五言律诗弄成 indice 的形式 以后方便输入 
five_poem_indice = []
for each_poem in five_poem:
  five_poem_indice.append([word2ix[word] for word in each_poem])

print(len(five_poem_indice))

print(five_poem_indice[:10])

59115
[[6731, 4770, 1787, 8118, 7577], [782, 782, 2063, 2031, 846], [2309, 2596, 6483, 2260, 7316], [7976, 6043, 258, 47, 7878], [3495, 70, 7113, 4839, 5237], [6134, 5522, 3052, 6863, 4426], [8087, 5274, 4426, 7686, 2354], [3611, 1175, 3052, 3951, 3878], [4102, 7739, 4007, 152, 2383], [2878, 1418, 6225, 2325, 5516]]


In [4]:
import torch
## 构造输入输出 连续采样，一句一句诗来取
def data_iter_consecutive(poem_indice, batch_size, num_step=4, device=None):
  if device == None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  
  epoch_size = len(poem_indice) // batch_size
  poem_indice = torch.tensor(poem_indice, dtype=torch.float32, device=device)
  indice = poem_indice[0:epoch_size * batch_size]

  for epoch in range(epoch_size):
    i = epoch * batch_size
    X = indice[i:i + batch_size]  ##现在得到的X是五个字。。但是输入只需要前四个字就ok
    # print(X)
    X = X[:, 0:4]
    Y = indice[i:i+batch_size]
    Y = Y[:, 1:5]
    yield X, Y


data_iter = data_iter_consecutive(five_poem_indice, 2)
i = 0
for X, Y in data_iter:
  if (i == 2):
    break
  i += 1
  print(X)
  print(Y)
  

tensor([[6731., 4770., 1787., 8118.],
        [ 782.,  782., 2063., 2031.]])
tensor([[4770., 1787., 8118., 7577.],
        [ 782., 2063., 2031.,  846.]])
tensor([[2309., 2596., 6483., 2260.],
        [7976., 6043.,  258.,   47.]])
tensor([[2596., 6483., 2260., 7316.],
        [6043.,  258.,   47., 7878.]])


## X, Y 都有了，现在需要把X弄成one-hot的形式

In [0]:

def one_hot(x, n_class, dtype=torch.float32): 
    # X shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res

def to_onehot(X, n_class):  
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]


## 初始化网络参数

In [0]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_hiddens = 256
vocab_size = len(ix2word)

rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)
num_step = 35 # 时间步数
batch_size = 2
state = None ## 隐藏状态



In [0]:
class RNNModel(nn.Module):

  def __init__(self, rnn_layer, vocab_size):
    super(RNNModel, self).__init__()
    self.rnn = rnn_layer
    self.hidden_size = rnn_layer.hidden_size 
    self.vocab_size = vocab_size

    self.dense = nn.Linear(self.hidden_size, vocab_size) ## 全联接层，最后输出用
    self.state = None 
  
  def forward(self, inputs, state):
    X = d2l.to_onehot(inputs, self.vocab_size)
    Y, self.state = self.rnn(torch.stack(X), state)

    ##全联接层 ，操作就是 output = Y.dot(Wy) + b  Wy.shape:(hidden_size, outputs_num)
    output = self.dense(Y.view(-1, Y.shape[-1]))##  写-1 或者 2 都可以！ 就是最后一维  按理说不用全联接层了，，rnn里面已经这样操作过了。 但这里加了一层
    ## output shape为：(num_steps * batch_size, vocab_size)
    return output, self.state



## 写一个预测函数

def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char, char_to_idx):
  state = None
  output = [char_to_idx[prefix[0]]] ##  先把第一个字符加入到结果里面

  for t in range(num_chars + len(prefix) - 1) :
    X = torch.tensor([output[-1]], device=device).view(1, 1) ## 构造出来X
    if state is not None: ## 不同设备转换～
        if isinstance(state, tuple): # LSTM, state:(h, c)  
            state = (state[0].to(device), state[1].to(device))
        else:   
            state = state.to(device)
    
    (y, state) = model(X, state)

    if (t < len(prefix) - 1):  ## 说明还有用户所输入的前缀，那么就要直接输出前缀数据就ok
      output.append(char_to_idx[prefix[t + 1]])
    else :
      # print("t=" + str(t))
      output.append(int(y.argmax(dim=1).item()))
    
  return "".join([idx_to_char[i] for i in output])



In [13]:
##随机权值预测一次
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分', 4, model, vocab_size, device, ix2word, word2ix)


'分礿特徨於'

In [0]:
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device, corpus_indices, idx_to_char, char_to_idx, num_epochs, 
                                  num_steps, lr, clipping_theta, batch_size, pred_period, pred_len, prefixes):
  loss = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr)

  model.to(device)
  state = None

  for epoch in range(num_epochs):
    l_sum, n, start = 0.0, 0, time.time()
    data_iter = data_iter_consecutive(corpus_indices, batch_size, device)

    for X, Y in data_iter:
      if state is not None:
        if isinstance (state, tuple): # LSTM, state:(h, c)  
          state = (state[0].detach(), state[1].detach())
        else:   
          state = state.detach()

      (output, state) = model(X, state)

      ## y 应该先转置 然后再view 成 跟output行数一样！
      y = torch.transpose(Y, 0, 1).contiguous().view(-1)
      l = loss(output, y.long())
      optimizer.zero_grad()##梯度清零
      l.backward()

      d2l.grad_clipping(model.parameters(), clipping_theta, device)
      optimizer.step()
      l_sum += l.item() * y.shape[0] ## shape[0] 是batch_size * num_step 因为求loss的时候 除以N了 

    if (epoch + 1) % pred_period == 0:
        print('epoch %d, time %.2f sec, loss %.8f' % (
                epoch + 1, time.time() - start, l_sum))
        for prefix in prefixes:
          print(' -', predict_rnn_pytorch(
                  prefix, pred_len, model, vocab_size, device, idx_to_char,
                  char_to_idx))


In [16]:
num_epochs, batch_size, lr, clipping_theta = 5, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 4, 4, ['红', '春', '愿', '此']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            five_poem_indice, ix2word, word2ix,
                            num_epochs, 4, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

torch.save(model, "./model.pkl")

epoch 4, time 246.81 sec, loss 810884.52627563
 - 红树共谁家
 - 春日登襄阳
 - 愿作扫芳尘
 - 此地一片石


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNNModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
